# 5 eda aeda back

### 1) Set functions
- ["Okt", "Kkma", "Komoran", "Mecab", "Hannanum"]

In [1]:
# # 분할을 원하지 않는 단어 모아두기
# no_tokenized = ['#Person#', '#Person1#', '#Person2#', '#Person3#', 
# '#Person4#', '#Person5#', '#Person6#', '#Person7#', '#사람1#',
# '#PhoneNumber#', '#Address#', '#PassportNumber#',
# '#DateOfBirth#','#SSN#','#CardNumber#','#CarNumber#','#Email#']

In [2]:
# from jamo import h2j, j2hcj

# def get_jongsung_TF(sample_text):
#     sample_text_list = list(sample_text)
#     last_word = sample_text_list[-1]
#     last_word_jamo_list = list(j2hcj(h2j(last_word)))
#     last_jamo = last_word_jamo_list[-1]

#     jongsung_TF = "T"

#     if last_jamo in ['ㅏ', 'ㅑ', 'ㅓ', 'ㅕ', 'ㅗ', 'ㅛ', 'ㅜ', 'ㅠ', 'ㅡ', 'ㅣ', 'ㅘ', 'ㅚ', 'ㅙ', 'ㅝ', 'ㅞ', 'ㅢ', 'ㅐ,ㅔ', 'ㅟ', 'ㅖ', 'ㅒ']:
#         jongsung_TF = "F"

#     return jongsung_TF
    

# # tmp 경로에 저장된 사용자 사전 명사에 새로운 단어 추가
# with open('../mecab-0.996-ko-0.9.2/mecab-ko-dic-2.1.1-20180720/user-dic/nnp.csv', "r", encoding='utf-8') as f:
#   user_dict = f.readlines()

# for word in no_tokenized:
#   jongsung_TF = get_jongsung_TF(word)
#   line = '{},*,*,*,NNP,*,{},{},*,*,*,*,*\n'.format(word, jongsung_TF, word)

#   user_dict.append(line)
  
# print(user_dict[-10:])

In [3]:
# # 재저장
# with open('../mecab-0.996-ko-0.9.2/mecab-ko-dic-2.1.1-20180720/user-dic/nnp.csv', 'w', encoding='utf-8') as f:
#   for line in user_dict:
#     f.write(line)

In [4]:
# load library
from koeda import EasyDataAugmentation, AEasierDataAugmentation
import pandas as pd
from tqdm.notebook import tqdm
import random
import time

# EDA
def augment_text_data_with_EDA(text_col, repetition):
    # 단어 바꾸기
    """입력된 문장에 대해서 EDA를 통해 데이터 증강"""
    eda = EasyDataAugmentation(
        morpheme_analyzer="Mecab"
        )
    
    random.seed(42)

    result = text_col.progress_apply(eda, p=(0.8, 0.01, 0.02, 0.01), repetition=repetition)
    print(result)
    
    return result

# AEDA
def augment_text_data_with_AEDA(text_col, repetition):
    # 단어 사이에 문자 추가하기
    """입력된 문장에 대해서 AEDA를 통해 데이터 증강"""
    aeda = AEasierDataAugmentation(
        morpheme_analyzer="Mecab",
        punctuations=[".", ",", "!", "?", ";", ":"]
    )

    random.seed(42)
    
    # result = text_col.progress_apply(aeda, p=0.05, repetition=repetition)
    result = []
    for i in tqdm(range(len(text_col))):
        time.sleep(0.01)
        a = aeda(text_col[i], p = 0.05, repetition=repetition)
        result.append(a)

    print(result)
    return result

In [5]:
# pandas tqdm 호출
tqdm.pandas() # tqdm의 pandas전용 메소드를 호출합니다.

### 2) application train data

In [6]:
# load original data
train = pd.read_csv('../data/back_result_train.csv')

In [7]:
# eda
train2 = train.sample(n=2500, random_state=42)
train2['dialogue'] = augment_text_data_with_EDA(train2['dialogue'], 1)
train2.to_csv('../data/back_mecab_eda_augmentation_made30000.csv', index = False)

  0%|          | 0/2500 [00:00<?, ?it/s]

5610     #Person1#: 안녕하세요! 트랜스퍼시픽항공 입니다. 어떻게 도와 드릴까요? #...
3047     #Person1#: 마이크, 내일 오후에 저 좀 도와주실 수 있나요? 오후 에시부터...
18927    #Person1#: 안녕, 마크. 어떻게 지내?#Person2#: 잘 지내, 신디....
1972     이득 #Person1#: 오늘은 무엇을 도와드릴까요, 앨리스? #Person2# 생...
13554    #Person1#: 무엇을 주문하시겠어요, 고객님?#Person2#: 중국 음식을 ...
                               ...                        
9052     #Person1#: 소피, 어젯밤에 Frank Jones를 주최한 퀴즈만듭니다쇼가 ...
4303     #Person1#: 역사 교수님께서 나에게 정치 분야의 직업을 고려해 보라고 하셨지...
16008    #Person1#: 크리스, 오늘어오전 11시에 할아버지, 할머니께 전화해야에하는 ...
10279    #Person1#: 여기는 11번 경찰서입니다. 무엇을 도와드릴까요? #Person...
2264     #Person1#: 마르쉐, 과학 강좌를 들어야 할까요? #Person2#: 네, ...
Name: dialogue, Length: 2500, dtype: object


In [8]:
# # aeda
# train3 = train.copy()
# train3['dialogue'] = augment_text_data_with_AEDA(train3['dialogue'], 1)
# train3.to_csv('../data/mecab_aeda_augmentation.csv')

In [25]:
# 자동화 안되서 분할 적용하기 위해 수동으로 적용
aeda = AEasierDataAugmentation(
    morpheme_analyzer="Mecab",
    punctuations=[".", ",", "!", "?", ";", ":"]
)
# sampling 
train_sample = train.sample(n=2500, random_state=42).reset_index(drop = True)

random.seed(42)
# result = text_col.progress_apply(aeda, p=0.05, repetition=repetition)
result = []
n = len(train_sample['dialogue'])
for i in tqdm(range(n)):
    a = aeda(train_sample['dialogue'][i], p = 0.05, repetition=1)
    result.append(a)

  0%|          | 0/2500 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [27]:
# 2탄
for i in tqdm(range(1867, n)):
    a = aeda(train_sample['dialogue'][i], p = 0.05, repetition=1)
    result.append(a)

  0%|          | 0/633 [00:00<?, ?it/s]

In [21]:
# # 3탄
# for i in tqdm(range(7268, 9000)):
#     a = aeda(train_sample['dialogue'][i], p = 0.05, repetition=1)
#     result.append(a)

In [22]:
# # 4탄
# for i in tqdm(range(19825, n)):
#     a = aeda(train_sample['dialogue'][i], p = 0.05, repetition=1)
#     result.append(a)

In [28]:
# 결과 비교
len(result)

2500

In [29]:
# 동일한 개수 확인
len(train_sample['dialogue'])

2500

In [30]:
# 저장
train3 = train_sample.copy()
train3['dialogue'] = result
train3.to_csv('../data/back_mecab_aeda_augmentation_made30000.csv', index = False)

In [31]:
# 통합 후 저장
train_merge = pd.concat([train, train2, train3])
train_merge.to_csv('../data/back_augment_train_merge_made30000.csv', index = False)